In [12]:
import pandas as pd
from itertools import cycle
from IPython.display import clear_output
import os
path_output = 'data/results/all.gff'
path_blast = 'data/results/'
pd.set_option('display.max_colwidth', -1)


In [13]:
#TEs
params = {}
params['LTR'] = {'file':'blast_clustered_ltr.csv.filtered'}
params['LINE'] = {'file':'blast_clustered_line.csv.filtered'}
params['SINE'] = {'file':'blast_clustered_sine.csv.filtered'}
params['TIR'] = {'file':'blast_clustered_tir.csv.filtered'}
params['MITE'] = {'file':'blast_clustered_mite.csv.filtered'}
params['LARD'] = {'file':'blast_clustered_lard.csv.filtered'}
params['helitron'] = {'file':'blast_clustered_helitron.csv.filtered'}
params['TRIM'] = {'file':'blast_clustered_trim.csv.filtered'}


In [14]:
#read blast output
dfs = []
for k,v in params.items():
    df = pd.read_csv(path_blast + v['file'], sep='\t')
    dfs.append(df)
df_all = pd.concat(dfs)
df_all.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
df_all = df_all.reset_index(drop=True)
print('initial:',len(df_all.index))

('initial:', 419056)


In [15]:
df_all['qseqid'].head()

0    TRIM_id_159_RLX_family30_TRIM_retrotransposon2395_chr03_60445_61898[type:TRIM;source:tephra][type:TRIM;source:tephra]                   
1    TRIM_id_2529_RLX_family32_TRIM_retrotransposon1146_chr00_33890312_33892052[type:TRIM;source:tephra][type:TRIM;source:tephra]            
2    TRIM_id_374_RLX_family96_TRIM_retrotransposon1026_chr00_29618301_29619085[type:TRIM;source:tephra][type:TRIM;source:tephra]             
3    TRIM_id_496_RLX_family96_TRIM_retrotransposon709_chr00_17949413_17950254[type:TRIM;source:tephra][type:TRIM;source:tephra]              
4    TRIM_id_6742_RLX_singleton_family10278_TRIM_retrotransposon719_chr00_18040087_18041240[type:TRIM;source:tephra][type:TRIM;source:tephra]
Name: qseqid, dtype: object

In [16]:
df_all['source'] = 'ST_annotation'
df_all['frame'] = '.'
df_all['score'] = '.'
df_all['strand'] = '+'
df_all['feature'] = 'TE'
splited_id = df_all['qseqid'].str.split('_')
df_all['auto_id'] = splited_id.str[0] + '_' + splited_id.str[1] + ':' + splited_id.str[2] + ';'
df_all['qseqid'] = df_all['qseqid'].str.split('[', 1).str[1]# + ';source_id:' + df_all['qseqid'].str.split('[', 1).str[0] 
df_all['qseqid'] = df_all['qseqid'].str.replace("\\]\\[",";")
df_all['qseqid'] = df_all['qseqid'].str.replace("]","")
df_all['qseqid'] = df_all['qseqid'].str.replace("[","")
df_all['qseqid'] = df_all['qseqid'].str.replace("[","")

In [17]:
df_all['qseqid'].head()

0    type:TRIM;source:tephra;type:TRIM;source:tephra
1    type:TRIM;source:tephra;type:TRIM;source:tephra
2    type:TRIM;source:tephra;type:TRIM;source:tephra
3    type:TRIM;source:tephra;type:TRIM;source:tephra
4    type:TRIM;source:tephra;type:TRIM;source:tephra
Name: qseqid, dtype: object

In [18]:
#order by chromosome and position
df_all['new_sstart'] = df_all[['sstart','send']].min(axis=1)
df_all['new_ssend'] = df_all[['sstart','send']].max(axis=1)
df_all['sstart'] = df_all['new_sstart']
df_all['send'] = df_all['new_ssend']
df_all = df_all.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df_all.sort_values(['sseqid', 'sstart'], inplace=True)

In [19]:
#delete dups annotation
count = 1
for k,v in df_all.iterrows():
    qs = {}
    for q in v.qseqid.split(";"):
        qs[q] = 1
    new_qseqid = ";".join(qs.keys())
    df_all.at[k, 'qseqid'] = new_qseqid
    df_all.at[k, 'qseqid'] = new_qseqid + ';unique_id:' + str(count)
    count += 1

In [20]:
df_all['qseqid'] = df_all['auto_id'] + df_all['qseqid']
df_all = df_all[['sseqid','source','feature','sstart','send','score','strand','frame','qseqid']]
#gff3 -> ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']

In [21]:
df_all['qseqid'].head()

0    TRIM_id:159;type:TRIM;source:tephra;unique_id:1 
1    TRIM_id:2529;type:TRIM;source:tephra;unique_id:2
2    TRIM_id:374;type:TRIM;source:tephra;unique_id:3 
3    TRIM_id:496;type:TRIM;source:tephra;unique_id:4 
4    TRIM_id:6742;type:TRIM;source:tephra;unique_id:5
Name: qseqid, dtype: object

In [23]:
df_all.to_csv(path_output, sep="\t", header=None, index=None)
path_output

'data/results/all.gff'